

# 第5课：蒙特卡洛方法（Monte Carlo Methods）

> with model to without model

---

## Part 0：Motivating Example - 均值估计（Mean Estimation）

### 问题背景

你有一个**未知分布** $p(x)$，但你可以从中**采样**（sampling）。现在你想估计它的**期望值**：

$$
\mu = \mathbb{E}_{x \sim p(x)}[x]
$$

问题是 —— 你**没有这个分布的显式形式**，无法解析地计算期望。怎么办？

---

### 蒙特卡洛估计的基本思想

如果你可以从该分布中采样 $x^{(1)}, x^{(2)}, \dots, x^{(n)}$，那么你就可以**用样本均值估计真实期望**：

$$
\hat{\mu}_n = \frac{1}{n} \sum_{i=1}^{n} x^{(i)} \approx \mu
$$

这是最简单的**蒙特卡洛估计**。它的核心逻辑是：

> “在不知道概率分布的情况下，通过反复采样，再用平均值代替期望值。”

---

### 示例：估计骰子的平均点数

我们用一颗标准六面骰子（均匀分布）模拟一个 $p(x)$，真值为：

$$
\mu = \mathbb{E}[x] = \frac{1+2+3+4+5+6}{6} = 3.5
$$

我们可以反复掷骰子并计算平均值：

* 掷 10 次：$\hat{\mu}_{10} = 3.6$
* 掷 100 次：$\hat{\mu}_{100} = 3.47$
* 掷 10000 次：$\hat{\mu}_{10000} = 3.501$

✅ 可以看到，随着采样数量 $n$ 增加，估计值 $\hat{\mu}_n \to \mu$。

---

### 总结：为什么这个例子重要？

| 方面       | 说明                                      |
| -------- | --------------------------------------- |
| 与蒙特卡洛的关系 | 强化学习中很多量（如 return）都是期望，无法解析计算，只能采样平均来估计 |
| 实质       | 蒙特卡洛就是：**“用样本均值代替期望”** 的通用方法            |
| 推广       | 在强化学习中，我们用多个 episode 得到的回报，来估计状态或动作值函数  |

---

### 对强化学习的启发

强化学习中的值函数（如 $V^\pi(s) = \mathbb{E}[G_t \mid s_t = s]$）是**期望回报**。如果我们不能用贝尔曼方程计算（因为环境模型未知），就可以：

> 像估计骰子期望一样，重复从状态 $s$ 出发运行策略 $\pi$，记录实际回报 $G_t$，用平均值估计 $V^\pi(s)$。

这就是蒙特卡洛方法的出发点。




## Part 1：通过例子介绍蒙特卡洛

在前几课中，我们通过 MDP 的状态转移概率 $P(s'|s,a)$ 来精确地计算值函数。但现实世界中，环境模型往往是**未知的**，这时我们可以：

> 通过**与环境交互得到的实际轨迹**，直接估计值函数 —— 这就是蒙特卡洛方法。

---

### 例子：玩 21 点（Blackjack）

我们不知道 21 点的全部概率模型，但可以让智能体玩很多局游戏，然后：

* 对每种状态 $s$，记录其在轨迹中的回报 $G_t$
* 估计状态值 $V(s)$ 为所有回报的平均值

例如，状态 `s = 18分 + 无A` 出现了 5 次，分别得到回报 0, 1, -1, 1, 1，那么估计值：

$$
V(s) = \frac{0 + 1 - 1 + 1 + 1}{5} = 0.4
$$

这就是蒙特卡洛评估（Monte Carlo Evaluation）的基本思想。



## Part 2：MC Basic 算法介绍（基于完整回合）

蒙特卡洛方法是一类**基于采样轨迹**、在**模型未知（model-free）环境下**学习策略的强化学习方法。

其核心思想是：

> **通过多次与环境交互生成完整轨迹（episodes），基于真实的“回报”估计值函数，并不断改进策略。**


![](./img/5_1.png)

---

### Step 1：Policy Evaluation（策略评估）

在给定一个固定策略 $\pi$ 下，目标是估计：

* **状态值函数** $V^\pi(s) = \mathbb{E}_\pi[G_t \mid s_t = s]$，或
* **动作值函数** $Q^\pi(s,a) = \mathbb{E}_\pi[G_t \mid s_t = s, a_t = a]$

#### 方法：

1. 重复运行策略 $\pi$，与环境交互生成完整的 episode（从起始状态到终止状态）；
2. 对 episode 中每一个状态 $s_t$，根据之后观察到的**总回报 $G_t$** 来更新估计；
3. 使用**样本平均**（Monte Carlo Estimate）更新状态值：

   * 若使用**第一次访问**（first-visit MC）方法：

     $$
     V(s) \leftarrow \text{平均所有第一次访问状态 } s \text{ 时的回报 } G_t
     $$

   * 若使用**每次访问**（every-visit MC）方法：

     $$
     V(s) \leftarrow \text{平均每次访问状态 } s \text{ 时的回报 } G_t
     $$

注意：**只能在 episode 完成之后才计算 $G_t$**，因为必须观测到“未来所有奖励”。

---

### Step 2：Policy Improvement（策略改进）

在完成策略评估之后，我们可以通过 **贪婪策略（greedy policy）** 来改进原有策略：

$$
\pi_{\text{new}}(s) = \arg\max_a Q^\pi(s,a)
$$

这个过程叫做：

> **蒙特卡洛控制（Monte Carlo Control）**：评估策略 + 基于估计值函数进行贪婪改进

可以与策略迭代的框架类比：

| 过程   | 经典 MDP 中                       | 蒙特卡洛版本                |
| ---- | ------------------------------ | --------------------- |
| 策略评估 | 用贝尔曼方程解 $V^\pi$                | 多次采样估计 $V^\pi, Q^\pi$ |
| 策略改进 | $\pi(s) = \arg\max Q^\pi(s,a)$ | 同上（贪婪策略）              |
| 收敛条件 | 迭代至收敛                          | 多轮采样后逼近最优策略           |

---

### MC Basic 算法小结

| 步骤         | 描述                                                   |
| ---------- | ---------------------------------------------------- |
| Episode 生成 | 多次从起始状态出发，运行当前策略 $\pi$，记录状态、动作和奖励序列                  |
| 回报计算       | 对每个状态/动作，计算它之后的 return $G_t$                         |
| 值函数估计      | 将所有 $G_t$ 对应到状态或状态动作对，做平均值估计                         |
| 策略改进       | 基于 $Q(s,a)$ 选取贪婪动作更新策略： $\pi(s) = \arg\max_a Q(s,a)$ |

---

### 注意事项

* MC 方法**只适用于 episodic 任务**（必须能终止，才能观测完整回报）；
* 由于依赖平均，所以数据效率相对低，需多次采样；
* 不需要环境模型，是典型的 **model-free** 方法；
* 评估与改进可以交替进行，从而逼近最优策略。





## Part 3：MC Basic 算法例子（策略评估 + 策略改进）

我们用一个简单的格子世界（Grid World）环境来说明蒙特卡洛基本方法的具体运行过程。

---

### 环境设定（Grid World）

* 状态空间 $S$：一个 4×4 的网格，共 16 个状态（编号 0 \~ 15）；
* 起始状态：每次从状态 0 开始；
* 终止状态：状态 15 是终点（terminal）；
* 动作空间 $A$：上、下、左、右（每个状态最多4个动作）；
* 奖励函数：
  * 每步奖励为 -1（惩罚步数，鼓励更快完成任务）；
  * 到达终点（状态 15）后 episode 结束；
* 策略 $\pi$：初始为**等概率随机策略**，即每个状态选择动作的概率都是 $\frac{1}{4}$。

---

### MC 策略评估过程（以状态值为例）

1. **收集 Episode**

   * 多次运行当前策略，从起始状态出发，直到到达终止状态；
   * 记录访问的状态序列 $s_0, s_1, \dots, s_T$，以及每步奖励 $r_1, r_2, \dots, r_T$。

2. **计算回报 $G_t$**

   * 采用折扣因子 $\gamma = 1$（无折扣）；
   * 对每个状态 $s_t$，回报为：

     $$
     G_t = r_{t+1} + r_{t+2} + \cdots + r_T
     $$

3. **更新值函数**

   * 如果采用 first-visit 方法，则仅在 episode 中首次访问某状态时更新：

     $$
     V(s) \leftarrow \text{访问过该状态的所有 } G_t \text{ 的平均}
     $$

---

### MC 策略改进过程

有了当前策略 $\pi$ 下的动作值函数估计 $Q(s,a)$ 后，我们可以执行如下策略改进：

$$
\pi_{\text{new}}(s) = \arg\max_a Q(s,a)
$$

这就得到一个**贪婪策略**，在状态 $s$ 下选择“看起来最好的动作”。

---

### 一个 Episode 的例子（伪数据）

假设从状态 0 出发，经历如下轨迹：

| 时间 $t$ | 状态 $s_t$ | 动作 $a_t$ | 奖励 $r_{t+1}$ |
| ------ | -------- | -------- | ------------ |
| 0      | 0        | ↓        | -1           |
| 1      | 4        | →        | -1           |
| 2      | 5        | ↓        | -1           |
| 3      | 9        | →        | -1           |
| 4      | 10       | ↓        | -1           |
| 5      | 14       | →        | -1           |
| 6      | 15 (终止)  | —        | —            |

则：

| 状态 $s_t$ | 第一次出现 | 回报 $G_t$ |
| -------- | ----- | -------- |
| 0        | ✅     | -6       |
| 4        | ✅     | -5       |
| 5        | ✅     | -4       |
| 9        | ✅     | -3       |
| 10       | ✅     | -2       |
| 14       | ✅     | -1       |

用这些 $G_t$ 对应每个状态更新 $V(s)$ 的估计。

---

### 整体流程小结

```python
Initialize V(s) arbitrarily
for each episode:
    generate an episode using current policy π
    for each state s visited in the episode:
        compute return G_t
        update V(s) as average of returns
```

若进一步估计 $Q(s,a)$，则每次记录的是 $(s_t, a_t, G_t)$ 三元组，对 $Q(s,a)$ 进行平均更新即可。

---

## 小结

| 步骤   | 内容                                   |
| ---- | ------------------------------------ |
| 策略评估 | 运行策略 π 得到多个 episode，基于回报 $G_t$ 更新值函数 |
| 策略改进 | 贪婪地更新策略：$\pi(s) = \arg\max_a Q(s,a)$ |
| 所需前提 | 需 episode 可终止；环境可采样；不依赖模型            |






## Part 4：MC Exploring Starts 算法（探索性起点）

### 背景问题：为什么不能直接用贪婪策略改进？

在 Part 3 中，我们采用如下策略改进方式：

$$
\pi(s) = \arg\max_a Q(s,a)
$$

但这存在一个**探索性不足**的问题：

* 如果某个状态–动作对 $(s, a)$ **从未出现**，那 $Q(s, a)$ 就没有值；
* 若只使用贪婪更新，可能会永远不尝试这个动作；
* 这样会导致“局部最优”甚至“训练失败”。

---

### 解决方案：Exploring Starts（探索性起点）

#### 思想核心：

> 在每个 episode 的开始阶段，**强制从一个随机的 (s, a) 对开始**，保证所有状态–动作对都有可能被探索。

这打破了“贪婪陷阱”，保证每个 $Q(s, a)$ 最终都有估计，从而可以正确优化策略。

---

### MC Exploring Starts 算法流程（控制问题）

目标：寻找一个最优策略 $\pi^*$，即使我们不知道环境模型。


![](./img/5_2.png)


#### 初始化：

* 对所有 $(s,a)$，初始化动作值函数 $Q(s,a)$（可设为0或随机）；
* 初始化策略 $\pi(s)$，使其贪婪地选择当前 $Q(s,a)$ 最大的动作；
* 记录每对 $(s,a)$ 的回报序列，或用累计平均更新。

---

#### 每次迭代过程：

1. **从一个随机的 (s, a) 起点开始 episode**

   * 选择任意状态 $s \in S$
   * 在该状态下强制执行一个动作 $a \in A$
   * 然后使用当前策略 $\pi$ 执行后续步骤，直到 episode 结束

2. **生成 episode**：

   得到状态–动作–奖励序列：

   $$
   (s_0, a_0, r_1), (s_1, a_1, r_2), \dots, (s_T, -, r_{T+1})
   $$

3. **计算每个 $(s_t, a_t)$ 对应的 return $G_t$**

4. **更新动作值函数 $Q(s,a)$**

   * first-visit MC：

     $$
     Q(s,a) \leftarrow \text{该对第一次出现时的所有回报的平均}
     $$

5. **策略改进**（greedy）：

   $$
   \pi(s) = \arg\max_a Q(s,a)
   $$

---

#### 算法伪代码（简化版）

```python
Initialize Q(s,a) arbitrarily for all s,a
Initialize π(s) = argmax_a Q(s,a)

Loop forever (or until convergence):
    Choose random s and a as the starting point
    Generate an episode following a, then π
    For each (s,a) in the episode:
        Compute return G_t
        Update Q(s,a) as average of returns
        Update π(s) = argmax_a Q(s,a)
```

---

### MC Exploring Starts 的性质

| 特点       | 说明                                             |
| -------- | ---------------------------------------------- |
| 保证探索     | 所有 $(s,a)$ 对都可能作为 episode 起点，从而保证 $Q(s,a)$ 可估计 |
| 可收敛到最优策略 | 在有限 MDP 中，随着采样次数增加，策略将收敛到最优策略（理论保证）            |
| 实用性差     | 很多环境中**无法随意设置起点**（如真实游戏中无法控制起点状态或初始动作）         |

---

### 小结

| 项目   | 内容                                           |
| ---- | -------------------------------------------- |
| 目标   | 找到最优策略 $\pi^*$ 和对应的 $Q^*$                    |
| 核心思想 | 用“随机起点 (s,a)” 强制探索，避免策略陷入局部                  |
| 实际限制 | 仅适用于可以任意指定起点的环境（如模拟器中）                       |
| 理论意义 | 提供了**第一个完整的、基于采样的最优控制方法**，为后续 ε-greedy 等算法奠基 |




## Part 5：MC ε-Greedy 算法介绍

---

### 背景问题回顾

我们之前介绍的 **Exploring Starts（ES）** 方法虽然理论完备，但有个重大缺陷：

> ❌ **现实环境中往往无法控制 episode 的起始状态和动作。**

为了解决这个问题，**ε-Greedy 策略**被引入：
即 **在每个状态下，大多数时候执行最优动作，小概率随机探索其他动作。**

---

### ε-Greedy 策略定义

在任意状态 $s$ 下：

$$
\pi(a \mid s) =
\begin{cases}
1 - \varepsilon + \frac{\varepsilon}{|A(s)|}, & \text{if } a = \arg\max_{a'} Q(s,a') \\
\frac{\varepsilon}{|A(s)|}, & \text{otherwise}
\end{cases}
$$

其中：

* $\varepsilon \in [0,1]$：探索率（越大表示越多探索）；
* $|A(s)|$：动作空间大小；
* 保证每个动作都有**非零概率被执行**（避免无法估计 $Q(s,a)$）。

---

### 算法核心思想

> 替代 Exploring Starts 的随机起点机制，通过 ε-Greedy 策略实现 **“持续探索”**。

---

### 算法流程（MC Control with ε-Greedy）

![](./img/5_3.png)

#### 初始化

* 对所有状态–动作对 $Q(s,a)$ 初始化（可设为0）；
* 初始化策略为 ε-greedy（基于当前 $Q$）；

---

#### 每轮训练过程

1. **从起始状态开始，生成完整 episode**

   * 每步根据当前 ε-greedy 策略选择动作 $a_t$
   * 得到序列 $(s_0, a_0, r_1), (s_1, a_1, r_2), \dots$

2. **对每个 $(s_t, a_t)$ 计算回报 $G_t$**

3. **更新动作值函数 $Q(s,a)$**

   * 使用 first-visit 或 every-visit 方法：

     $$
     Q(s,a) \leftarrow \text{该对的历史回报的平均}
     $$

4. **改进策略**

   * 用最新的 $Q$ 更新 ε-greedy 策略（即重新贪婪 + 探索）：

     $$
     \pi(s) = \text{ε-greedy w.r.t. } Q(s,a)
     $$

---

#### 伪代码示意

```python
Initialize Q(s,a) arbitrarily
Set ε > 0 (e.g., 0.1)

Loop forever (or for N episodes):
    Generate episode following ε-greedy π
    For each (s,a) in the episode:
        Compute G_t
        Update Q(s,a) ← average of all G_t for (s,a)
    For each state s:
        π(s) ← ε-greedy based on current Q(s,a)
```

---

### ε-Greedy MC 方法优点

| 优点                   | 说明                                                    |
| -------------------- | ----------------------------------------------------- |
| 不需要 Exploring Starts | 不要求能控制初始状态–动作对，适用于真实环境                                |
| 持续探索                 | 每个动作都有概率被执行，从而所有 $Q(s,a)$ 都可被估计                       |
| 收敛性保证                | 若所有 $(s,a)$ 被访问无穷多次，值函数估计将收敛到 $Q^\pi$，策略将收敛到最优策略（概率1） |

---

### 常见设置建议

| 参数            | 建议                                            |
| ------------- | --------------------------------------------- |
| $\varepsilon$ | 初始 0.1，或逐步减小（如 $\varepsilon_t = \frac{1}{t}$） |
| 初始化           | $Q(s,a) = 0$ 一般足够                             |
| 策略更新频率        | 每轮 episode 结束后即可更新一次策略                        |

---

### 总结

| 特性                    | 内容                               |
| --------------------- | -------------------------------- |
| 是否需要模型                | ❌ No，model-free                  |
| 是否需要 episode 完整       | ✅ Yes（MC 要求完整轨迹）                 |
| 是否自动探索                | ✅ Yes（通过 ε-greedy 策略）            |
| 是否收敛到最优策略             | ✅ Yes，只要所有 $(s,a)$ 被无限次访问        |
| 与 Exploring Starts 区别 | 不强制初始状态–动作随机，而是**每一步都允许一定概率的探索** |




## Part 6：MC ε-Greedy 算法例子

---

### 环境说明：简化版 GridWorld

设定一个 3x3 的网格世界（9 个状态，编号 0\~8）：

* 起始状态：总从状态 0 开始；
* 终止状态：状态 8（右下角）为终止状态；
* 动作空间：上、下、左、右；
* 边界动作将原地不动；
* 每走一步奖励为 -1，终止状态无奖励；
* 折扣因子 $\gamma = 1$；
* 初始策略为 ε-greedy 随机策略（ε = 0.1）；
* 初始化所有 $Q(s,a) = 0$。

---

### 一个 Episode 示例

假设某轮采样中，我们得到如下 episode（含回报计算）：

| 时刻 $t$ | 状态 $s_t$ | 动作 $a_t$ | 奖励 $r_{t+1}$ | 回报 $G_t$ |
| ------ | -------- | -------- | ------------ | -------- |
| 0      | 0        | →        | -1           | -4       |
| 1      | 1        | ↓        | -1           | -3       |
| 2      | 4        | ↓        | -1           | -2       |
| 3      | 7        | →        | -1           | -1       |
| 4      | 8        | —        | —            | 0        |

使用 **first-visit MC** 方法对首次出现的 $(s_t, a_t)$ 进行更新：

```python
Q(0, →) ← -4
Q(1, ↓) ← -3
Q(4, ↓) ← -2
Q(7, →) ← -1
```

---

### 多轮迭代后的策略更新

在每轮 episode 后，策略根据 ε-greedy 方法进行更新。例如，某状态下的 Q 值：

```python
Q(4, ↑) = -3
Q(4, ↓) = -2   ← 最优
Q(4, ←) = -4
Q(4, →) = -3
```

则更新后的 ε-greedy 策略如下（动作空间共4个）：

* 最优动作 ↓ 的概率：

  $$
  \pi(↓|4) = 1 - \varepsilon + \frac{\varepsilon}{4} = 0.925
  $$

* 其他动作（↑, ←, →）概率为：

  $$
  \pi(a|4) = \frac{\varepsilon}{4} = 0.025
  $$

从而策略不会被“锁死”，仍保持一定概率尝试非最优动作，以免陷入局部最优。

---

### 多次训练的收敛趋势

随着 episode 越来越多：

* $Q(s,a)$ 会趋于 $Q^\pi(s,a)$；
* 策略 π 会逐渐向最优策略靠拢；
* 探索（ε）可以逐渐减小，如 ε-decay；
* 训练最终会收敛到某个 ε-最优策略（若 ε > 0）或最优策略（ε → 0）。

---

## 小结

| 要素   | 内容                                   |
| ---- | ------------------------------------ |
| 初始策略 | ε-greedy 策略，初始 $Q(s,a)=0$            |
| 策略评估 | 每个 episode 后，使用 MC 方法更新 $Q(s,a)$     |
| 策略改进 | 更新 ε-greedy 策略，选择当前估计值最高的动作          |
| 关键特性 | 不需要模型、不需要 Exploring Starts，但持续探索所有动作 |
| 示例结果 | 多轮采样后 $Q(s,a)$ 精确估计 → 策略收敛到最优策略      |



# Q1 什么是model-free？


## 原话回顾：

> “在前几课中，我们通过 MDP 的状态转移概率 $p(s'|s,a)$ 来精确地计算值函数。但现实世界中，环境模型往往是**未知的**。”

---

## 什么是“环境模型”？

在马尔可夫决策过程（MDP）中，环境模型指的是你知道以下两件事：

1. **状态转移概率**：

   $$
   p(s'|s,a) = \text{在状态 } s \text{ 执行动作 } a \text{ 后转移到状态 } s' \text{ 的概率}
   $$

2. **即时奖励函数**：

   $$
   r(s,a) = \text{在状态 } s \text{ 执行动作 } a \text{ 时的期望奖励}
   $$

> 如果你拥有这两个函数，就能完整地模拟环境，不需要真实地与环境交互就能“推演”出所有策略的效果。

---

## “环境模型未知”是什么意思？

现实中，强化学习智能体往往面对一个**黑箱环境**，也就是说：

* 你**不知道** $p(s'|s,a)$，即不知道一个动作具体会把你带到哪里；
* 你**不知道** $r(s,a)$，即不知道执行动作能获得什么奖励；
* 你能做的就是：在状态 $s$ 下选择一个动作 $a$，观察环境反馈（新状态 $s'$、奖励 $r$）。

这样的情况就是所谓的 **模型未知（model-free）**。

---

## 举个现实例子

想象你在玩一个你从未接触过的电子游戏：

* 你不知道每个按钮（动作）会产生什么样的后果（转移）；
* 你不知道做出某个动作会不会加分（奖励）；
* 你只能通过**不断试玩并观察结果**，慢慢地学会“哪个动作更好”。

这就是一个**模型未知的环境**。

---

## 蒙特卡洛方法的动机

在这种情况下：

* 无法用贝尔曼方程（因为你不知道 $p$ 和 $r$）；
* 但你可以玩几局游戏、记录结果（轨迹），然后**直接用回报 $G_t$** 来估计值函数。

例如：玩 10 局游戏，每次从状态 $s$ 开始，最终获得的回报是：

$$
G_t^{(1)}, G_t^{(2)}, \dots, G_t^{(10)}
$$

你就可以估计值函数为：

$$
V(s) \approx \frac{1}{10} \sum_{i=1}^{10} G_t^{(i)}
$$

这就是典型的**蒙特卡洛估计**。

---

## 总结：环境模型未知的含义

| 项目       | 含义                            |                        |
| -------- |-------------------------------|------------------------|
| “环境模型已知” | 你知道  $p(s'                    | s,a)  和 r(s,a)$，可以计算期望 |
| “环境模型未知” | 你只能观察到环境反馈，**不能计算期望，只能采样学习**  |                        |
| 蒙特卡洛的作用  | 在模型未知的情况下，通过与环境交互的轨迹来**估计值函数** |                        |


- model-free algorithm directly estimates action values.

